<a href="https://colab.research.google.com/github/EddyGiusepe/Spam_classifier_using_Python/blob/main/spam_classification_ML_End_to_End_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spam classification Machine Learning | End-to-End Deployment | spam classifier using Python

Data Scientist Jr.: Dr.Eddy Giusepe Chirinos Isidro


\\
Here I will study how to build and implement ``end-to-end`` Spam Classification Model using NLP Machine Learning. I will study and show you how to create a WORD CLOUD in this tutorial.


Here, too, we will learn how to create, use Machine Learning Model Pickle file. 


This study was based on the following links:

* [Spam classifier using Python](https://www.youtube.com/watch?v=inp6h0HDN74&list=PLlUZLZydkS79ctBEkAqeNj5k0RNvAvvD7&index=6).

* [Automated spam e-mail detection model | using commom NLP tasks](https://www.analyticsvidhya.com/blog/2021/06/automated-spam-e-mail-detection-modelusing-common-nlp-tasks/)

In [1]:
# Importamos a nossas bibliotecas

import pandas as pd
import numpy as np
import pickle
import nltk
import re
import csv

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.metrics import accuracy_score, fbeta_score, classification_report
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
ss = SnowballStemmer('english')
ps = PorterStemmer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [109]:
# Carregando nossos Dados

# with open('/content/drive/MyDrive/3_EDDY_ISH_TECNOLOGIA/4_ML_inside_of _MANTIS/Spam_classification_ML/spam.csv', mode='r', encoding='ISO-8859-1') as arquivo:
#   arquivo_csv = csv.reader(arquivo, delimiter=',')
#   linhas = 0
#   for coluna in arquivo_csv:
#     if linhas == 0:
#       print(f'Colunas:  {" ".join(coluna)}')
#       linhas += 1
#     else:
#       print(f'\t{coluna[0]} é o {coluna[1]}')
#       linhas += 1 

#   print(f'lidas {linhas} linhas.')      




In [2]:
# Ver vídeo para ler um arquivo .csv --> https://www.youtube.com/watch?v=K1RLuCp_LvI

msg_df = pd.read_csv('/content/drive/MyDrive/3_EDDY_ISH_TECNOLOGIA/4_ML_inside_of _MANTIS/Spam_classification_ML/spam.csv',
                     sep=',', encoding='ISO-8859-1', header=0, usecols=['class', 'message'])

msg_df.head(20)
#display(msg_df)

,class,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [3]:
msg_df.shape

(5572, 2)

In [4]:
# stop words

stop;

In [9]:
# Guia de como trocar os nomes da coluna --> https://www.youtube.com/watch?v=k_C5szI5V7Q
# Aqui vamos a trocar o nome da primeira coluna

msg_df.rename(columns={'class': 'label'}, inplace=True)

In [10]:
msg_df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
msg_df.describe()

,label,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [17]:
# Tambṕem --> msg_df.groupby('label').describe().T

msg_df.groupby('label').describe()

message                                                               
        count unique                                                top freq
label                                                                       
ham      4825   4516                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4

In [18]:
msg_df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64